## Key Takeaways
- To check parameters, I could use `parameters()` method or `named_parameters()` method on `nn.Module` class

---

In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline


[Learn the Basics](intro.html) ||
[Quickstart](quickstart_tutorial.html) ||
[Tensors](tensorqs_tutorial.html) ||
[Datasets & DataLoaders](data_tutorial.html) ||
[Transforms](transforms_tutorial.html) ||
**Build Model** ||
[Autograd](autogradqs_tutorial.html) ||
[Optimization](optimization_tutorial.html) ||
[Save & Load Model](saveloadrun_tutorial.html)

# Build the Neural Network

Neural networks comprise of layers/modules that perform operations on data.
The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace provides all the building blocks you need to
build your own neural network. Every module in PyTorch subclasses the [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules (layers). This nested structure allows for
building and managing complex architectures easily.

In the following sections, we'll build a neural network to classify images in the FashionMNIST dataset.


In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get Device for Training
We want to be able to train our model on a hardware accelerator like the GPU or MPS,
if available. Let's check to see if [torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html)
or [torch.backends.mps](https://pytorch.org/docs/stable/notes/mps.html) are available, otherwise we use the CPU.



In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


## Define the Class
We define our neural network by subclassing ``nn.Module``, and
initialize the neural network layers in ``__init__``. Every ``nn.Module`` subclass implements
the operations on input data in the ``forward`` method.



In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of ``NeuralNetwork``, and move it to the ``device``, and print
its structure.



In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model's ``forward``,
along with some [background operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Do not call ``model.forward()`` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of each output.
We get the prediction probabilities by passing it through an instance of the ``nn.Softmax`` module.



In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4])


---

In [7]:
X.shape

torch.Size([1, 28, 28])

In [8]:
X.size()

torch.Size([1, 28, 28])

In [9]:
logits.size()

torch.Size([1, 10])

In [10]:
type(logits)

torch.Tensor

In [11]:
pred_probab

tensor([[0.1021, 0.1022, 0.1041, 0.0942, 0.1059, 0.0917, 0.1056, 0.0992, 0.0959,
         0.0993]], grad_fn=<SoftmaxBackward0>)

In [12]:
pred_probab.size()

torch.Size([1, 10])

In [13]:
pred_probab.argmax(0)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [14]:
pred_probab.argmax(1)

tensor([4])

--------------




## Model Layers

Let's break down the layers in the FashionMNIST model. To illustrate it, we
will take a sample minibatch of 3 images of size 28x28 and see what happens to it as
we pass it through the network.



In [15]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten
We initialize the [nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (
the minibatch dimension (at dim=0) is maintained).



In [16]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear
The [linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
is a module that applies a linear transformation on the input using its stored weights and biases.




In [17]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU
Non-linear activations are what create the complex mappings between the model's inputs and outputs.
They are applied after linear transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) between our
linear layers, but there's other activations to introduce non-linearity in your model.



In [18]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-6.7518e-01, -8.1046e-02,  1.7746e-01,  2.6872e-02, -4.7729e-01,
         -3.6040e-01,  1.8452e-01, -8.3831e-01,  5.0096e-01,  4.1481e-01,
          1.1174e-02,  1.0342e-01, -1.8364e-01,  5.3025e-01, -4.3740e-01,
         -5.3109e-02,  8.0715e-02, -4.9063e-03,  3.0413e-01, -1.1203e-01],
        [-2.0355e-01, -2.2715e-01,  2.1499e-01, -1.6413e-02, -3.4866e-01,
         -4.5462e-01,  2.3085e-01, -6.1009e-01,  5.3687e-01,  2.5918e-02,
         -2.4044e-01, -7.2935e-02, -4.7096e-02,  6.3025e-01, -6.4654e-01,
         -2.2372e-01,  2.3227e-01,  1.9520e-01,  1.7682e-01,  2.6699e-01],
        [-3.2846e-01,  2.1275e-01,  2.2385e-01, -1.3065e-02, -5.7871e-01,
         -4.1726e-01, -1.8999e-02, -2.3997e-01,  4.6906e-01,  1.4591e-01,
         -9.3828e-02, -1.6244e-01, -3.1518e-01,  7.3014e-01, -4.9871e-01,
          1.4788e-01,  7.1612e-04,  1.4017e-01,  1.2933e-01,  9.0222e-02]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000e+00, 0.0000e+00, 1.7746e-01, 2.6

### nn.Sequential
[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) is an ordered
container of modules. The data is passed through all the modules in the same order as defined. You can use
sequential containers to put together a quick network like ``seq_modules``.



In [19]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### nn.Softmax
The last linear layer of the neural network returns `logits` - raw values in [-\infty, \infty] - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) module. The logits are scaled to values
[0, 1] representing the model's predicted probabilities for each class. ``dim`` parameter indicates the dimension along
which the values must sum to 1.



In [20]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## Model Parameters
Many layers inside a neural network are *parameterized*, i.e. have associated weights
and biases that are optimized during training. Subclassing ``nn.Module`` automatically
tracks all fields defined inside your model object, and makes all parameters
accessible using your model's ``parameters()`` or ``named_parameters()`` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.




In [21]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0229, -0.0325,  0.0146,  ...,  0.0330, -0.0157,  0.0220],
        [-0.0116,  0.0095, -0.0221,  ...,  0.0335, -0.0151, -0.0194]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0277, 0.0281], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0213,  0.0434,  0.0257,  ...,  0.0278, -0.0024, -0.0361],
        [ 0.0257,  0.0243,  0.0325,  ..., -0.0400, -0.0269,  0.0221]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Si

--------------




In [22]:
a = model.parameters()

In [23]:
type(a)

generator

In [24]:
for i in a:
    print(i)
    break

Parameter containing:
tensor([[-0.0229, -0.0325,  0.0146,  ...,  0.0330, -0.0157,  0.0220],
        [-0.0116,  0.0095, -0.0221,  ...,  0.0335, -0.0151, -0.0194],
        [-0.0309,  0.0049,  0.0353,  ..., -0.0145,  0.0250,  0.0041],
        ...,
        [ 0.0010,  0.0193, -0.0207,  ..., -0.0155, -0.0174, -0.0212],
        [-0.0159, -0.0288,  0.0307,  ..., -0.0283, -0.0198, -0.0037],
        [-0.0023,  0.0095,  0.0338,  ..., -0.0092, -0.0263,  0.0197]],
       requires_grad=True)


In [25]:
type(model)

__main__.NeuralNetwork

## Further Reading
- [torch.nn API](https://pytorch.org/docs/stable/nn.html)

